In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.image import resize

In [ ]:
model = tf.keras.models.load_model("Trained_model.h5")  #Trained_model.h5 is the model that has been created.

In [15]:
classes = ['blues', 'classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']

In [20]:
def load_and_preprocess_data(file_path, target_shape=(150, 150)):
    data = []
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    chunk_duration = 4  
    overlap_duration = 2  
                
    chunk_samples = chunk_duration * sample_rate
    overlap_samples = overlap_duration * sample_rate
                
    num_chunks = int(np.ceil((len(audio_data) - chunk_samples) / (chunk_samples - overlap_samples))) + 1
                
    for i in range(num_chunks):
        start = i * (chunk_samples - overlap_samples)
        end = start + chunk_samples
                    
        chunk = audio_data[start:end]
                    
        mel_spectrogram = librosa.feature.melspectrogram(y=chunk, sr=sample_rate)
                    
        mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
        data.append(mel_spectrogram)
    
    return np.array(data)

In [9]:
from IPython.display import Audio
file_path = "./genre/rock/rock.00015.au"
y, sr = librosa.load(file_path, sr=44100)
Audio(data=y, rate=sr)

In [ ]:
#Processing Test File
X_test = load_and_preprocess_data(file_path)

### Model Prediction

In [ ]:
def model_prediction(X_test):
    y_pred = model.predict(X_test)
    predicted_categories = np.argmax(y_pred,axis=1)
    unique_elements, counts = np.unique(predicted_categories, return_counts=True)
    max_count = np.max(counts)
    max_elements = unique_elements[counts == max_count]
    return max_elements[0]

In [85]:
c_index = model_prediction(X_test)
print(f"Model Prediction :: Music Genre --> {classes[c_index]}")

3/3 [==============================] - 0s 46ms/step
Model Prediction :: Music Genre --> rock


### Thus the model created identifies the audio file belonging to the genre "rock"